In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv

In [36]:
load_dotenv(dotenv_path=find_dotenv())
from financial.model import Model
import financial.data as fd
from financial.io.file.cache import FileCache

In [37]:
cache = os.environ["CACHE"]
model = os.environ["MODEL"]
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache+"/"))

In [38]:
model = Model.from_file("rnn_BLK_2025_single", path=os.environ["MODEL"], model_type="financial.momentum.models.kerasAdvanced.RecurrentModel")
print(type(model))  # debería dar RecurrentModel
model.model.compile(
    loss="mean_squared_error",
    optimizer="adam",
    metrics=["mae"]
)
predictions = model.get_data(datastore, start_index="2010-01-01", end_index="2023-12-31")
print(predictions)

<class 'financial.momentum.models.kerasAdvanced.RecurrentModel'>
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Date
2010-05-13   -0.011536
2010-05-14   -0.011536
2010-05-17   -0.011536
2010-05-18   -0.011536
2010-05-19   -0.011536
                ...   
2023-12-22   -0.011536
2023-12-26   -0.011536
2023-12-27   -0.011536
2023-12-28   -0.011536
2023-12-29   -0.011536
Length: 3432, dtype: float32


In [39]:
model_rforest = Model.from_file('transformer_BLK_2025_single', path = os.environ["MODEL"])
print(type(model))
predictions_rforest = model_rforest.get_data(datastore, start_index="2010-01-01", end_index="2023-12-31")
print(predictions_rforest)

<class 'financial.momentum.models.kerasAdvanced.RecurrentModel'>
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Date
2010-05-13   -0.01117
2010-05-14   -0.01117
2010-05-17   -0.01117
2010-05-18   -0.01117
2010-05-19   -0.01117
               ...   
2023-12-22   -0.01117
2023-12-26   -0.01117
2023-12-27   -0.01117
2023-12-28   -0.01117
2023-12-29   -0.01117
Length: 3432, dtype: float32


In [55]:
from keras.models import load_model
from lxml import etree
import json
from financial.momentum.models.kerasAdvanced import RecurrentModel
file = "kerascnn_GOOG_2025_single_f"
path = os.environ["MODEL"]
xml_path = os.path.join(path, filename + ".xml")
base_path = os.path.join(path, file)

# 1. Cargar metadatos
xml = Model.metadata_from_xml_file(base_path)
model_id, sources, target = xml

# 2. Cargar hiperparámetros
with open(base_path + ".hyperparameters.json") as f:
    hyperparameters = json.load(f)

# 3. Cargar modelo Keras desde archivo (.keras o .h5)
model_path = base_path + ".keras"
if not os.path.exists(model_path):
    model_path = base_path + ".h5"
model = load_model(model_path, compile=False)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

# 4. Devolver una instancia real de RecurrentModel
final_model=RecurrentModel(model_id, sources, target, model=model, hyperparameters=hyperparameters)

final_model.model.summary()
weights = model.get_weights()
import numpy as np
print("Número de capas con pesos:", len(weights))
print("STD de las últimas capas:", np.std(weights[-1]))

FileNotFoundError: [Errno 2] No such file or directory: '/home/manidmt/TFG/OTRI/models/keras/kerascnn_GOOG_2025_single_f.xml'

In [41]:
X = final_model.get_input_data(datastore, start_index="2010-01-01", end_index="2023-12-31")
X_reshaped = final_model.reshape_input(X)
print("Input shape:", X.shape)
print("Reshaped input shape:", X_reshaped.shape)

print("Varianza de entrada:", np.var(X_reshaped, axis=(1, 2)))
print("Valores únicos en salida:", np.unique(final_model.predict(X)))

Input shape: (3522, 90)
Reshaped input shape: (3522, 90, 1)
Varianza de entrada: [       nan        nan        nan ... 0.00117106 0.00115942 0.00114931]
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Valores únicos en salida: [-0.01738538 -0.01738117 -0.01738107 ...  0.01310874  0.01320393
         nan]


In [56]:
file = "kerascnn_GOOG_2025_single_f"
path = os.environ["MODEL"]
base_path = os.path.join(path, file)
model_path = base_path + ".keras"
model = load_model(model_path)
model = load_model(model_path, compile=False)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()
weights = model.get_weights()
import numpy as np
print("Número de capas con pesos:", len(weights))
print("STD de las últimas capas:", np.std(weights[-1]))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 90, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 45, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 45, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 22, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2816)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         2,817 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,777 (108.50 KB)

 Trainable params: 27,777 (108.50 KB)

 Non-trainable params: 0 (0.00 B)

Número de capas con pesos: 6
STD de las últimas capas: 0.0


In [43]:
file_format = "h5"
filepath = os.path.join(os.environ["MODEL"], filename + "." + file_format)
model=load_model(filepath)
file_format = "xml"
filepath = os.path.join(os.environ["MODEL"], filename + "." + file_format)
inputDescriptor = fd.DataDescriptor.from_xml_string(filepath)

ParseError: not well-formed (invalid token): line 1, column 0 (<string>)